In [5]:
!pip install pettingzoo
#install all dependencies
!pip install pettingzoo[all]
!pip install pymunk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached multi-agent-ale-py-0.1.11.tar.gz (551 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pygame-2.3.0.tar.gz (12.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [81 lines of output]
      
      
      WARNING, No "Setup" File Exists, Running "buildconfig/config.py"
      Using WINDOWS configuration...
      
      Making dir :prebuilt_downloads:
      Downloading... https://www.libsdl.org/release/SDL2-devel-2.24.2-VC.zip 19b1e9f5bcf700a402bb924c0194c96596283c32
      Unzipping :prebuilt_downloads\SDL2-devel-2.24.2-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_image/release/SDL2_image-devel-2.0.5-VC.zip 137f86474691f4e12e76e07d58d5920c8d844d5b
      Unzipping :prebuilt_downloads\SDL2_image-devel-2.0.5-VC.zip:
      Downloading... https://github.com/libsdl-org/SDL_ttf/releases/download/release-2.20.1/SDL2_ttf-devel-2.20.1-VC.zip 371606aceba450384428fd2852f73d2f6290b136
      Unzipping :prebuilt_downloads\SDL2_ttf-devel-2.20.1-VC.zip:
      Downloading... https://github.com/libsdl-org/SDL_mixer/releases/downlo

Defaulting to user installation because normal site-packages is not writeable


# Initializing Environments
在 PettingZoo 中使用环境与在 Gymnasium 中使用环境非常相似。您可以通过以下方式初始化环境：

In [6]:
from pettingzoo.butterfly import pistonball_v6
env = pistonball_v6.env()

环境通常可以通过创建时的参数进行高度配置，即：

In [7]:
from pettingzoo.butterfly import cooperative_pong_v5

cooperative_pong_v5.env(ball_speed=18, left_paddle_speed=25,
right_paddle_speed=25, cake_paddle=True, max_cycles=900, bounce_randomness=False)

# Interacting With Environments

可以使用与 Gymnasium 类似的界面与环境进行交互：

In [8]:
from pettingzoo.butterfly import cooperative_pong_v5

env = cooperative_pong_v5.env(render_mode="human")
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        # this is where you would insert your policy
        action = env.action_space(agent).sample()

    env.step(action)
env.close()

常用的方法有：

agent_iter(max_iter=2**63) 返回一个迭代器，该迭代器 yields the current agent 。 
**当环境中的所有代理完成或max_iter （步骤已执行）时，它终止。**

last(observe=True) **返回当前能够采取行动的代理的观察、奖励、完成和信息。**
返回的奖励是智能体自上次行动以来收到的累积奖励。如果将observe设置为 False，则不会计算观察值，并且将返回 None 来代替它。请注意，完成单个代理并不意味着环境已完成。

reset()重置环境并设置它以供第一次调用时使用。该方法必须在任何其他方法之前调用。

step(action)接受并执行环境中代理的操作，自动将控制权切换到下一个代理。

## Additional Environment API
PettingZoo 将游戏建模为智能体环境循环(AEC) 游戏，因此可以支持多智能体 RL 可以考虑的任何游戏，从而允许出现极其奇怪的情况。因此，我们的 API 包含您可能不需要但在您需要时非常重要的较低级别的函数和属性。不过，它们的功能用于实现上面的高级函数，因此包含它们只是代码分解的问题。

agents ：所有当前代理的名称列表，通常为整数。这些可能会随着环境的进展而改变（即可以添加或删除代理）。

num_agents ：代理列表的长度。

agent_selection与当前选择的代理相对应的环境属性，可以对其采取操作。

observation_space(agent)检索特定代理的观察空间的函数。对于特定的代理 ID，此空间不应更改。

action_space(agent)检索特定代理的操作空间的函数。对于特定的代理 ID，此空间不应更改。

terminations ：调用时每个当前代理的终止状态的字典，按名称键入。 last()访问该属性。请注意，可以在此字典中添加或删除代理。返回的字典如下所示：

truncations ：调用时每个当前代理的截断状态的字典，按名称键入。 last()访问该属性。请注意，可以在此字典中添加或删除代理。返回的字典如下所示：

infos ：每个当前代理的信息字典，按名称键入。每个代理的信息也是一个字典。请注意，可以在此属性中添加或删除代理。 last()访问该属性。返回的字典如下所示：

observe(agent) ：返回代理当前可以进行的观察。 last()调用此函数。

rewards ：调用时每个当前代理的奖励的字典，按名称键入。奖励上一步后产生的瞬时奖励。请注意，可以在此属性中添加或删除代理。 last()不直接访问此属性，而是将返回的奖励存储在内部变量中。奖励结构如下：

{0:[first agent's reward], 1:[second agent's reward] ... n-1:[nth agent's reward]}

seed(seed=None) ：重新seed环境。 reset()必须在seed()之后、 step()之前调用。

render() ：使用初始化时指定的渲染模式从环境返回渲染帧。在渲染模式为'rgb_array'情况下，返回一个 numpy 数组，而使用'ansi'则返回打印的字符串。在human模式下无需调用render() 。

close() ：关闭渲染窗口。


## Optional API Components

虽然基本 API 不需要，但大多数下游包装器和实用程序依赖于以下属性和方法，并且应将它们添加到新环境中，除非在无法添加一个或多个的特殊情况下。

possible_agents ：环境可能生成的所有 possible_agents 的列表。相当于观察和行动空间中的代理列表。这不能通过播放或重置来更改。

max_num_agents ：possible_agents 列表的长度。

observation_spaces ：每个代理的观察空间的字典，按名称键入。这不能通过播放或重置来更改。

action_spaces ：每个代理的操作空间的字典，按名称键入。这不能通过播放或重置来更改。

state() ：返回环境当前状态的全局观察。并非所有环境都支持此功能。

state_space ：环境全局观察的空间。并非所有环境都支持此功能。

## Notable Idioms 值得注意的习语

### 检查整个环境是否完成
当代理agent 终止terminated 或截断truncated时，它将从agents中删除，因此当环境完成时， agents将是一个空列表。
**这意味着 not env.agents 表示环境完成。**

### Unwrapping an environment

如果您有一个包装环境，并且您希望在所有包装器层下获取未包装环境（以便您可以手动调用函数或更改环境的某些底层方面），则可以使用.unwrapped属性。如果环境已经是基础环境，则.unwrapped属性将仅返回其自身。

In [ ]:
from pettingzoo.butterfly import knights_archers_zombies_v10

base_env = knights_archers_zombies_v10.env().unwrapped

## Variable Numbers of Agents (Death)

代理可以在环境过程中死亡和生成。如果一个代理死亡，那么它在terminated字典中的条目将设置为True ，它成为下一个选定的代理（或在另一个也被终止或截断的代理之后），并且它采取的操作必须为None 。

执行此空步骤后，代理将从agents和其他可更改属性中删除。代理生成只需将其附加到agents和其他可更改的属性（它已经位于可能的代理和操作/观察空间中），并在某个时刻使用 agent_iter 转换到它即可完成。

## Environment as an Agent

在某些情况下，将介质与环境行为分开有助于研究。这可以通过将环境视为代理来实现。我们鼓励在 env.agents 中调用环境参与者env ，并让它采取None作为操作。

## Raw Environments
默认情况下，环境被包装在一些轻量级包装器中，这些包装器处理错误消息并确保在错误使用的情况下采取合理的行为（即在重置之前进行非法移动或步进）。然而，这些增加了非常少量的开销。如果您想创建一个没有它们的环境，可以使用每个模块中包含的raw_env()构造函数来实现：

In [ ]:
env = knights_archers_zombies_v10.raw_env(<environment parameters>)